In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
import urllib
from urllib.parse import urlparse
import codecs
import re

import pandas as pd



In [2]:
!pip install tqdm

     |████████████████████████████████| 71kB 4.5MB/s eta 0:00:011


In [11]:
%%capture
from tqdm import tqdm
from tqdm.notebook import tqdm, trange
import time    # to be used in loop iterations


In [23]:
url = "https://everydaysexism.com/"

In [24]:
page = urlopen(url)
html = page.read().decode("utf-8", errors='strict')
soup = BeautifulSoup(html, "html.parser")

In [40]:
# print(soup)

In [53]:
names = []
dates = []
entries = []
tags = []

def bs_setup(page_number):
    if page_number == 1:
        url = "https://everydaysexism.com/"
    else:
        url = "https://everydaysexism.com/page/{}".format(page_number)
        
    page = urlopen(url)
    html = page.read().decode("utf-8", errors='strict')
    soup = BeautifulSoup(html, "html.parser")

    return soup

def lists_setup(start, end): 
    for num in trange(start,end): #includes first number, excludes second #trange so that i can have a progress bar...
        time.sleep(10)
        
        soup = bs_setup(num)
        
        for title in soup.find_all('h2'):
            names.append(title.find('a').text)

        for date in soup.find_all('span', {'class': "entry-date"}):
            dates.append(date.text)

        for entry in soup.find_all('section', {'class': "entry-content"}):
            entries.append(entry.text)

        for tag in soup.find_all('span', {'class': 'tag-links'}):
            list_ = []
            list_.append(re.sub('Tags: ', '', tag.text))
            tags.append(list_)

lists_setup(1,10)

In [54]:
df = pd.DataFrame(list(zip(names, dates, entries,tags)), columns =['Name', 'Date', 'Post', 'Tags'])

In [56]:
df.shape

(90, 4)

In [40]:
rows = []

def bs_setup(page_number):
    if page_number == 1:
        url = "https://everydaysexism.com/"
    else:
        url = "https://everydaysexism.com/page/{}".format(page_number)
        
    page = urlopen(url)
    html = page.read().decode("utf-8", errors='strict')
    soup = BeautifulSoup(html, "html.parser")

    return soup

def lists_setup(start, end): 
    for num in trange(start,end): #includes first number, excludes second #trange so that i can have a progress bar...
        time.sleep(10) #right spot?
        soup = bs_setup(num)
        
        for title in soup.find_all('h2'):
            row = []
            
            row.append(title.find('a').text)
            row.append(title.find_next('span', {'class': "entry-date"}).text)
            row.append(title.find_next('section', {'class': "entry-content"}).text)
            row.append(title.find_next('span', {'class': 'tag-links'}).text)
            
            rows.append(row)

lists_setup(1,3)

In [49]:
df2 = pd.DataFrame(rows, columns =['Name', 'Date', 'Post', 'Tags'])

In [50]:
df2.head(3)

,Name,Date,Post,Tags
0,M,9th November 2020,\nTwice now I have had food delivered to me wh...,"Tags: Food delivery, Home"
1,Ramya,9th November 2020,"\nDuring a presentation today, my Male Colleag...",Tags: Workplace
2,Yara,9th November 2020,\nA few months ago I (16) was sexually assault...,Tags: Public space


In [52]:
# print(soup)